# Importation des librairies PyTorch et torchvision

Ce script commence par importer les librairies PyTorch et torchvision, qui sont utilisées pour le développement d'applications de machine learning et de vision par ordinateur.

- `torch` : Librairie principale pour l’apprentissage automatique et le traitement des tensors.
- `torchvision` : Librairie complémentaire à PyTorch contenant des datasets, des transformations, et des modèles pré-entraînés utiles pour les tâches de vision par ordinateur.

In [ ]:
import torch
from torchvision import datasets, transforms

# Chargement et préparation du dataset MNIST

Ce script charge et prépare le dataset MNIST, un célèbre dataset de digits manuscrits. MNIST est souvent utilisé pour les tâches de classification.

### Préparation des transformations

Les transformations appliquées au dataset sont composées comme suit :
- `transforms.ToTensor()` : Convertit les images en tensors.
- `transforms.Normalize((0.5,), (0.5,))` : Normalise les images en les centrant autour de 0.5 avec une échelle de 0.5.


In [ ]:
# Définition des transformations pour les données MNIST
transform = transforms.Compose(
    [transforms.ToTensor(),  # Convertit les images en tensors PyTorch
     transforms.Normalize((0.5,), (0.5,))]  # Normalise les données pour les centrer autour de 0 et normalise à l’échelle de 0.5
)

# Chargement de l'ensemble de données MNIST pour l'entraînement
sub_mnist_dataset1 = datasets.MNIST(
    "data",  # Chemin où les données seront stockées/téléchargées
    train=True,  # Utilise les données d'entraînement
    download=True,  # Télécharge les données si elles ne sont pas encore présentes
    transform=transform  # Applique les transformations définies plus haut
)

# Extraction d'un sous-ensemble des données : indices allant de 1200 à 1800 (données d'entraînement)
sub_mnist_dataset1.data = sub_mnist_dataset1.data[1200:1800]  # Sélection des lignes entre 1200 et 1800
sub_mnist_dataset1.target = sub_mnist_dataset1.targets[1200:1800]  # Sélection des targets correspondants

# Chargement du deuxième sous-ensemble de l'ensemble de données MNIST pour l'entraînement
sub_mnist_dataset2 = datasets.MNIST(
    "data",
    train=True,
    download=True,
    transform=transform
)

# Extraction d'un sous-ensemble des données : indices allant de 600 à 1200 (données d'entraînement)
sub_mnist_dataset2.data = sub_mnist_dataset2.data[600:1200]  # Sélection des lignes entre 600 et 1200
sub_mnist_dataset2.target = sub_mnist_dataset2.targets[600:1200]  # Sélection des targets correspondants

# Chargement des données de test de l'ensemble MNIST
sub_mnist_dataset_test = datasets.MNIST(
    "data",
    train=False,  # Utilise les données de test
    download=True,
    transform=transform
)

# Chargement des données d'entraînement avec DataLoader pour le premier sous-ensemble
train_loader1 = torch.utils.data.DataLoader(
    sub_mnist_dataset1,  # Premier sous-ensemble d'entraînement
    batch_size=50,  # Taille des mini-batchs pour l'entraînement
    shuffle=True  # Mélange des données à chaque époque
)

# Chargement des données d'entraînement avec DataLoader pour le second sous-ensemble
train_loader2 = torch.utils.data.DataLoader(
    sub_mnist_dataset2,  # Second sous-ensemble d'entraînement
    batch_size=50,
    shuffle=True
)

# Chargement des données de test avec DataLoader
test_loader = torch.utils.data.DataLoader(
    sub_mnist_dataset_test,  # Données de test
    batch_size=50,
    shuffle=True  # Mélange des données à chaque époque
)


# Définition du modèle Neural Network

Ce script définit un réseau de neurones convolutif simple pour la classification d'images.

### Classe `Net`
La classe `Net` hérite de `nn.Module`, ce qui lui permet d'implémenter les différentes couches du modèle.

- **Conv1** : Première couche de convolution avec 1 canal d'entrée et 4 canaux de sortie, une taille de filtre de 3x3 et un pas de 1.
- **Conv2** : Deuxième couche de convolution avec 4 canaux d'entrée et 2 canaux de sortie, également avec une taille de filtre de 3x3 et un pas de 1.
- **fc1** : Première couche entièrement connectée (FC) prenant les 50 valeurs de sortie de la dernière couche convolutive et les transformant en 32 neurones.
- **fc2** : Deuxième couche entièrement connectée, prenant les 32 neurones de la couche précédente et les transformant en 10 classes (output final).

### Fonction `forward`
La fonction `forward` est responsable de la propagation des données à travers le réseau :
- **conv1** : Applique une convolution suivie d’une activation ReLU.
- **max_pool2d** : Applique un max-pooling avec une taille de 2x2.
- **conv2** : Applique une deuxième convolution suivie d’une activation ReLU.
- **max_pool2d** : Applique un deuxième max-pooling avec une taille de 2x2.
- **flatten** : Met les données convolutives en un vecteur plat de taille appropriée.
- **fc1** : Applique une couche fully connectée avec une activation ReLU.
- **fc2** : Applique la couche fully connectée finale.
- **log_softmax** : Applique une fonction log-softmax pour obtenir les probabilités des différentes classes.

In [ ]:
import torch.nn as nn  # Importation du module pour les couches de réseaux neuronaux de PyTorch
import torch.nn.functional as F  # Importation des fonctions pour les calculs dans les couches

# Définition du modèle de réseau neuronal (class Net) qui hérite de nn.Module
class Net(nn.Module):
    def __init__(self, seed=0):
        super(Net, self).__init__()  # Appel du constructeur de la classe parent nn.Module

        torch.manual_seed(seed)  # Fixation du seed pour reproductibilité des résultats

        # Première couche convolutionnelle : 1 entrée, 4 sorties, taille du noyau 3x3
        self.conv1 = nn.Conv2d(1, 4, 3, 1)

        # Deuxième couche convolutionnelle : 4 entrées, 2 sorties, taille du noyau 3x3
        self.conv2 = nn.Conv2d(4, 2, 3, 1)

        # Première couche de connexion linéaire (fully connected) : 50 entrées, 32 sorties
        self.fc1 = nn.Linear(50, 32)

        # Deuxième couche de connexion linéaire (fully connected) : 32 entrées, 10 sorties
        self.fc2 = nn.Linear(32, 10)

    # Fonction de passage en avant (forward pass)
    def forward(self, x):
        # Application de la première couche convolutionnelle suivie d'une activation ReLU
        x = F.relu(self.conv1(x))

        # Application d'une opération de max pooling sur les cartes de convolution
        x = F.max_pool2d(x, 2, 2)

        # Application de la deuxième couche convolutionnelle suivie d'une activation ReLU
        x = F.relu(self.conv2(x))

        # Application d'une autre opération de max pooling
        x = F.max_pool2d(x, 2, 2)

        # Ré-organisation des dimensions pour passer à la couche fully connected
        x = x.view(x.size(0), -1)  # Flattening des dimensions des feature maps

        # Passage à la première couche fully connected avec activation ReLU
        x = F.relu(self.fc1(x))

        # Passage à la deuxième couche fully connected sans activation car la sortie est utilisée en softmax
        x = self.fc2(x)

        # Softmax pour normaliser les sorties au niveau des probabilités
        return F.log_softmax(x, dim=1)  # Retourne les probabilités logaires


# Fonctions d'agrégation et de mise à jour des paramètres du modèle

## 1. `average_model_parameters(models, average_weights)`
Cette fonction prend en entrée :
- **models** : Une liste de modèles PyTorch (`nn.Module`) dont on souhaite obtenir les paramètres.
- **average_weights** : Un tenseur contenant les poids de chaque modèle, utilisés pour pondérer les contributions des différents modèles.

Elle retourne :
- **avg_params** : Une liste contenant les paramètres moyens des modèles, pondérés par `average_weights`.

### Description :
- **avg_params** : Initialise une liste de zéro de la même taille que les paramètres du premier modèle.
- **Boucle** : Pour chaque modèle dans la liste, elle ajoute les paramètres pondérés à la liste `avg_params`.
- **Sortie** : Une liste contenant les paramètres moyennés de tous les modèles.


In [ ]:
# Fonction pour calculer les paramètres moyens à partir de plusieurs modèles
def average_model_parameters(models: list[nn.Module], average_weights: torch.Tensor):
    """
    Calcule les paramètres moyens des modèles fournis, en pondérant les paramètres selon des poids spécifiques.

    Args:
        models (list[nn.Module]): Liste de modèles PyTorch.
        average_weights (torch.Tensor): Tenseur contenant les poids associés à chaque modèle.

    Returns:
        list[torch.Tensor]: Liste des paramètres moyens résultants.
    """

    # Initialisation d'une liste pour stocker les paramètres moyens
    avg_params = [torch.zeros_like(p) for p in models[0].parameters()]

    # Boucle sur chaque modèle dans la liste
    for nb_model, model in enumerate(models):
        # Boucle sur chaque paramètre du modèle
        for i, p in enumerate(model.parameters()):
            # Accumule les paramètres pondérés en fonction des poids fournis
            avg_params[i] += p * average_weights[nb_model]

    return avg_params


# Fonction pour mettre à jour les paramètres d'un modèle avec les paramètres moyens
def update_model(model, avg_params):
    """
    Met à jour les paramètres d'un modèle avec les paramètres moyens donnés.

    Args:
        model (nn.Module): Modèle PyTorch dont les paramètres vont être mis à jour.
        avg_params (list[torch.Tensor]): Liste des paramètres moyens à utiliser pour la mise à jour.
    """

    # Boucle sur chaque paramètre du modèle
    for i, p in enumerate(model.parameters()):
        # Met à jour les paramètres du modèle avec les moyennes calculées
        p.data = avg_params[i]


## Fonction `train_batch(model, data, target, optimizer)`
Cette fonction effectue une étape de formation sur un batch spécifique de données.

### Description :
- **model.train()** : Met le modèle en mode d'entraînement.
- **optimizer.zero_grad()** : Met à zéro les gradients des paramètres du modèle.
- **output** : Calcule la sortie du modèle pour les données.
- **loss** : Calcul du loss avec la fonction `nll_loss` (Negative Log Likelihood Loss).
- **loss.backward()** : Calcule les gradients des paramètres en fonction de la perte.
- **optimizer.step()** : Applique les mises à jour des paramètres du modèle selon les gradients.
- **Retour** : Le modèle entraîné après la formation d’un batch.

In [ ]:
# Optimiseur Adam pour l'entraînement du modèle
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_batch(model, data, target, optimizer):
    """
    Entraîne le modèle sur un batch de données.

    Args:
        model (nn.Module): Le modèle PyTorch à entraîner.
        data (torch.Tensor): Les données d'entrée.
        target (torch.Tensor): Les cibles/étiquettes.
        optimizer (torch.optim.Optimizer): L'optimiseur utilisé pour la descente de gradient.

    Returns:
        nn.Module: Le modèle entraîné.
    """
    model.train()  # Passe le modèle en mode d'entraînement
    optimizer.zero_grad()  # Met les gradients à zéro
    output = model(data)  # Passer les données à travers le modèle
    loss = F.nll_loss(output, target)  # Calcule la perte
    loss.backward()  # Effectue la rétropropagation pour calculer les gradients
    optimizer.step()  # Met à jour les paramètres du modèle
    return model  # Retourne le modèle entraîné

def train(models, train_loaders, average_weights, epochs=5, rounds=2):
    """
    Entraîne plusieurs modèles sur leurs sous-ensembles de données.

    Args:
        models (list[nn.Module]): Liste des modèles PyTorch à entraîner.
        train_loaders (list[DataLoader]): Liste de chargeurs de données pour chaque modèle.
        average_weights (torch.Tensor): Poids pour les modèles lors de l'average des paramètres.
        epochs (int): Nombre d'époques pour l'entraînement.
        rounds (int): Nombre de tours d'entraînement.

    Returns:
        None
    """

    # Initialisation des optimiseurs pour chaque modèle
    optimizers = [torch.optim.SGD(model.parameters(), lr=1e-1) for model in models]

    for round in range(rounds):  # Parcourt chaque tour d'entraînement
        epoch_losses = [[] for _ in range(len(models))]  # Stocke les pertes pour chaque modèle

        for epoch in range(epochs):  # Parcourt chaque époque
            for i, loader in enumerate(train_loaders):  # Parcourt les chargeurs de données
                for data, target in loader:
                    model = models[i]  # Modèle correspondant au chargeur
                    optimizer = optimizers[i]  # Optimiseur correspondant au modèle

                    model.train()  # Passe le modèle en mode d'entraînement
                    optimizer.zero_grad()  # Met les gradients à zéro

                    output = model(data)  # Passe les données à travers le modèle
                    loss = F.nll_loss(output, target)  # Calcule la perte
                    loss.backward()  # Effectue la rétropropagation pour calculer les gradients
                    optimizer.step()  # Met à jour les paramètres du modèle

                    epoch_losses[i].append(loss.item())  # Stocke la perte pour cette itération

        # Calcul des paramètres moyens à la fin de chaque tour
        avg_params = average_model_parameters(models, average_weights)

        for i, model in enumerate(models):  # Affiche et met à jour chaque modèle après le round
            avg_loss = sum(epoch_losses[i]) / len(epoch_losses[i])
            print(f"Model {i}, Round {round}, Loss: {avg_loss}")
            update_model(model, avg_params)  # Met à jour les paramètres du modèle avec l'average


## Seed aléatoire

### Configuration des hyperparamètres
- **`epochs`** : Définie le nombre d’époques à effectuer pour chaque round d’entraînement.
- **`rounds`** : Nombre total de rounds au cours desquels la formation est effectuée.


In [ ]:
rounds = 50
epochs = 5

## Initialisation des modèles et chargeurs de données

### model1 et model2 : Création de deux instances du modèle Net, qui représente un réseau de neurones convolutif simple.
### models : Liste contenant les deux modèles à entraîner.
### train_loaders : Liste des DataLoader, contenant les sous-ensembles d’entraînement pour chaque batch.
### average_weights : Poids pour l’agrégation des paramètres, dans cet exemple, ils sont équilibrés à 0.5 pour chaque modèle.

In [ ]:
# Définition de deux modèles identiques
model1 = Net()  # Instanciation du premier modèle basé sur la classe Net
model2 = Net()  # Instanciation du deuxième modèle basé sur la classe Net

# Stockage des modèles dans une liste pour la gestion simultanée
models = [model1, model2]  # Liste des modèles PyTorch à entraîner

# Liste des chargeurs de données correspondant à chaque modèle (train_loader1, train_loader2)
train_loaders = [train_loader1, train_loader2]

# Poids associés à chaque modèle pour l'average des paramètres
average_weights = torch.tensor([0.5, 0.5])  # Poids uniformes pour les deux modèles


### Exécution de la formation
## train() : Fonction principale pour l’entraînement des modèles. Elle prend en entrée les modèles, les chargeurs de données, les poids d’agrégation, ainsi que le nombre d’époques et de rounds.
## Reproductibilité : Le seed aléatoire est fixé via torch.manual_seed() pour garantir que les résultats soient reproductibles.

In [ ]:
# Appel de la fonction de formation avec les modèles, les chargeurs de données, les poids d'average,
# et les paramètres d'entraînement tels que le nombre d'époques et le nombre de rounds
train(models, train_loaders, average_weights, epochs=epochs, rounds=10)


Model 0, Round0, loss: 2.3023083329200746
Model 1, Round0, loss: 2.3022481083869932
Model 0, Round1, loss: 2.295469951629639
Model 1, Round1, loss: 2.296802508831024
Model 0, Round2, loss: 2.291091299057007
Model 1, Round2, loss: 2.293339455127716
Model 0, Round3, loss: 2.2837798953056336
Model 1, Round3, loss: 2.2879899978637694
Model 0, Round4, loss: 2.2720150351524353
Model 1, Round4, loss: 2.277334813276927
Model 0, Round5, loss: 2.2526381572087604
Model 1, Round5, loss: 2.260564299424489
Model 0, Round6, loss: 2.2251285513242087
Model 1, Round6, loss: 2.23793021440506
Model 0, Round7, loss: 2.196560871601105
Model 1, Round7, loss: 2.21383931239446
Model 0, Round8, loss: 2.1595162908236185
Model 1, Round8, loss: 2.189759639898936
Model 0, Round9, loss: 2.119257962703705
Model 1, Round9, loss: 2.1465267380078634


## Calcul de l’accuracy
### correct = 0 : Initialisation de la variable pour compter les prédictions correctes.
### for data, target in test_loader : Boucle sur les batches du test_loader, où les données de test sont fournies.
### output = models(data) : Applique le modèle 2 (la seconde instance du modèle) sur le batch de données.
### pred = output.argmax(dim=1, keepdim=True) : Calcule la classe prédite en utilisant l’argmax sur les sorties du modèle.

In [ ]:
# Passage des modèles en mode évaluation pour désactiver les calculs du gradient et les couches de batchnorm
models[0].eval()  # Passe le premier modèle en mode évaluation
models[1].eval()  # Passe le deuxième modèle en mode évaluation

correct = 0  # Initialisation du compteur de prédictions correctes

# Boucle sur les données du test_loader
for data, target in test_loader:
    # Calcul de la sortie du deuxième modèle
    output = models[1](data)

    # Prédiction avec argmax sur la sortie
    pred = output.argmax(dim=1, keepdim=True)  # Retourne la classe ayant le plus grand score

    # Calcule les prédictions correctes (comparer les prédictions avec les targets)
    correct += pred.eq(target.view_as(pred)).sum().item()  # Compte les prédictions correctes

# Affiche l'exactitude moyenne sur l'ensemble des données de test
print(f"Accuracy: {correct / len(test_loader.dataset)}")


Accuracy: 0.1089


## Initialisation des modèles avec Seed Fixé
### **`model1 = Net(seed=42)`**
- **`Net(seed=42)`** : Instancie un modèle de type `Net` avec un seed spécifique fixé à 42. Cela signifie que les poids initiaux du modèle sont déterminés de manière reproductible en fonction du seed donné.

In [ ]:
# Instanciation du premier modèle avec un seed fixe pour la reproductibilité
model1 = Net(seed=42)  # Création du premier modèle basé sur la classe Net avec un seed défini

# Instanciation du deuxième modèle avec le même seed pour assurer la reproductibilité des résultats
model2 = Net(seed=42)  # Création du deuxième modèle basé sur la classe Net avec un seed défini

# Liste contenant les deux modèles PyTorch pour les utiliser simultanément dans l'entraînement
models = [model1, model2]  # Liste des deux modèles

# Liste des chargeurs de données correspondants pour chaque sous-ensemble de données d'entraînement
train_loaders = [train_loader1, train_loader2]  # Chargeur de données pour le premier sous-ensemble
                                           # Chargeur de données pour le deuxième sous-ensemble

# Poids utilisés pour l'average des paramètres des modèles (les deux modèles ont le même poids)
average_weights = torch.tensor([0.5, 0.5])  # Poids uniformes pour les modèles lors de la fusion des paramètres


## Exécution de la Formation des Modèles

In [ ]:
train(models, train_loaders, average_weights, epochs=epochs, rounds=20)

## Évaluation des modèles sur les données d’entraînement

- **`models[0].eval()` et `models[1].eval()`** : Met les deux modèles en mode de validation (`eval()`), où les calculs liés à la normalisation ne sont pas activés.
- **`correct = 0`** : Initialise un compteur pour les prédictions correctes.
- **`for data, target in train_loader1`** : Boucle sur les données d’entraînement.
- **`output = models[1](data)`** : Applique le modèle 1 à chaque batch.
- **`pred = output.argmax(dim=1, keepdim=True)`** : Prédictions sous forme de tensor avec la classe prédite.
- **`pred.eq(target.view_as(pred)).sum().item()`** : Compte les prédictions correctes.
- **`correct / len(train_loader1.dataset)`** : Calcule l’accuracy sur tout le dataset d’entraînement.

In [ ]:
# Passage des modèles en mode évaluation pour désactiver les calculs du gradient et les couches de batchnorm
models[0].eval()  # Passe le premier modèle en mode évaluation
models[1].eval()  # Passe le deuxième modèle en mode évaluation

# Initialisation du compteur de prédictions correctes
correct = 0

# Boucle sur les données du train_loader1
for data, target in train_loader1:
    # Calcul de la sortie du deuxième modèle
    output = models[1](data)

    # Prédiction avec argmax sur la sortie
    pred = output.argmax(dim=1, keepdim=True)  # Retourne la classe ayant le plus grand score

    # Calcule les prédictions correctes (comparer les prédictions avec les targets)
    correct += pred.eq(target.view_as(pred)).sum().item()  # Compte les prédictions correctes

# Affiche l'exactitude moyenne sur l'ensemble des données d'entraînement
print(f"Accuracy: {correct / len(train_loader1.dataset)}")


Accuracy: 0.5333333333333333
